In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, PowerTransformer, QuantileTransformer,Normalizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from matplotlib import pyplot as plt
import seaborn as sns

import numpy as np

In [2]:
team_df = pd.read_csv('../../Basketball Reference Stat Scraper/team_stats/all.csv', index_col=0)
game_df = pd.read_csv('../../Basketball Reference Stat Scraper/GameLogs/all.csv', index_col=0)

In [3]:
# game_df = game_df[game_df['Year'] = 1980]
game_df = game_df.dropna()

team_df.Team = team_df.Team.str.strip('*')

In [4]:
# d = {}
#
# for i in team_df.Team.drop(team_df[team_df.Team.str.contains('League')].index).unique():
#     d[i] = ''
# d

In [5]:
team_mapping = {'Boston Celtics': 'BOS',
                 'Los Angeles Lakers': 'LAL',
                 'Philadelphia 76ers': 'PHI',
                 'Seattle SuperSonics': 'SEA',
                 'Phoenix Suns': 'PHX',
                 'Atlanta Hawks': 'ATL',
                 'Milwaukee Bucks': 'MIL',
                 'Kansas City Kings': 'KCK',
                 'Houston Rockets': 'HOU',
                 'San Antonio Spurs': 'SAS',
                 'Washington Bullets': 'WSB',
                 'New York Knicks': 'NYK',
                 'Portland Trail Blazers': 'POR',
                 'Indiana Pacers': 'IND',
                 'Cleveland Cavaliers': 'CLE',
                 'San Diego Clippers': 'SDC',
                 'New Jersey Nets': 'NJN',
                 'Chicago Bulls': 'CHI',
                 'Denver Nuggets': 'DEN',
                 'Utah Jazz': 'UTA',
                 'Golden State Warriors': 'GSW',
                 'Detroit Pistons': 'DET',
                 'Dallas Mavericks': 'DAL',
                 'Los Angeles Clippers': 'LAC',
                 'Sacramento Kings': 'SAC',
                 'Miami Heat': 'MIA',
                 'Orlando Magic': 'ORL',
                 'Minnesota Timberwolves': 'MIN',
                 'Charlotte Hornets': 'CHH',
                 'Toronto Raptors': 'TOR',
                 'Vancouver Grizzlies': 'VAN',
                 'Washington Wizards': 'WAS',
                 'Memphis Grizzlies': 'MEM',
                 'New Orleans Hornets': 'NOH',
                 'New Orleans/Oklahoma City Hornets': 'NOK',
                 'Charlotte Bobcats': 'CHO',
                 'Oklahoma City Thunder': 'OKC',
                 'Brooklyn Nets': 'BRK',
                 'New Orleans Pelicans': 'NOP'}

In [6]:
team_df['Team_Abb'] = team_df.Team.map(team_mapping)

team_df = team_df.drop(columns=['Unnamed: 0'], errors='ignore')

abc = []
for i in team_df.columns:
    abc.append(f'Season_{i}')

team_df.columns = abc



In [7]:
actual = pd.merge(left=game_df[['Tm_1_Name', 'Tm_2_Name', 'Date', 'Score_Diff', 'W/L', 'Year']], right=team_df,
                  how='left', left_on=['Tm_1_Name', 'Year'], right_on=['Season_Team_Abb', 'Season_Year'])
actual

,Tm_1_Name,Tm_2_Name,Date,Score_Diff,W/L,Year,Season_Team,Season_G,Season_MP,Season_FG,...,Season_SRS,Season_ORtg,Season_DRtg,Season_NRtg,Season_Pace,Season_FTr,Season_3PAr,Season_TS%,Season_Year,Season_Team_Abb
0,NYK,CHI,2014-10-29,-24,2,2015,New York Knicks,82.0,242.1,37.0,...,-9.50,99.9,110.0,-10.1,91.2,0.234,0.240,0.508,2015.0,NYK
1,NYK,CLE,2014-10-30,5,1,2015,New York Knicks,82.0,242.1,37.0,...,-9.50,99.9,110.0,-10.1,91.2,0.234,0.240,0.508,2015.0,NYK
2,NYK,CHO,2014-11-02,3,1,2015,New York Knicks,82.0,242.1,37.0,...,-9.50,99.9,110.0,-10.1,91.2,0.234,0.240,0.508,2015.0,NYK
3,NYK,WAS,2014-11-04,-15,2,2015,New York Knicks,82.0,242.1,37.0,...,-9.50,99.9,110.0,-10.1,91.2,0.234,0.240,0.508,2015.0,NYK
4,NYK,DET,2014-11-05,-3,2,2015,New York Knicks,82.0,242.1,37.0,...,-9.50,99.9,110.0,-10.1,91.2,0.234,0.240,0.508,2015.0,NYK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91933,DET,CLE,2006-04-12,23,1,2006,Detroit Pistons,82.0,241.8,35.5,...,6.24,110.8,103.1,7.7,86.8,0.297,0.221,0.535,2006.0,DET
91934,DET,TOR,2006-04-14,-5,2,2006,Detroit Pistons,82.0,241.8,35.5,...,6.24,110.8,103.1,7.7,86.8,0.297,0.221,0.535,2006.0,DET
91935,DET,NYK,2006-04-16,6,1,2006,Detroit Pistons,82.0,241.8,35.5,...,6.24,110.8,103.1,7.7,86.8,0.297,0.221,0.535,2006.0,DET
91936,DET,MIL,2006-04-17,-20,2,2006,Detroit Pistons,82.0,241.8,35.5,...,6.24,110.8,103.1,7.7,86.8,0.297,0.221,0.535,2006.0,DET


In [8]:
actual = pd.merge(left=actual, right=team_df, how='inner', left_on=['Tm_2_Name', 'Year'],
                  right_on=['Season_Team_Abb', 'Season_Year'], )
# actual.loc[actual['Season_Team_x'].isna().any(),:]


# actual.columns
actual

,Tm_1_Name,Tm_2_Name,Date,Score_Diff,W/L,Year,Season_Team_x,Season_G_x,Season_MP_x,Season_FG_x,...,Season_SRS_y,Season_ORtg_y,Season_DRtg_y,Season_NRtg_y,Season_Pace_y,Season_FTr_y,Season_3PAr_y,Season_TS%_y,Season_Year_y,Season_Team_Abb_y
0,NYK,CHI,2014-10-29,-24,2,2015,New York Knicks,82.0,242.1,37.0,...,2.54,107.5,104.3,3.2,92.8,0.304,0.269,0.536,2015.0,CHI
1,NYK,CHI,2014-12-18,-6,2,2015,New York Knicks,82.0,242.1,37.0,...,2.54,107.5,104.3,3.2,92.8,0.304,0.269,0.536,2015.0,CHI
2,NYK,CHI,2015-03-28,-31,2,2015,New York Knicks,82.0,242.1,37.0,...,2.54,107.5,104.3,3.2,92.8,0.304,0.269,0.536,2015.0,CHI
3,CHO,CHI,2014-12-03,-7,2,2015,NaN,NaN,NaN,NaN,...,2.54,107.5,104.3,3.2,92.8,0.304,0.269,0.536,2015.0,CHI
4,CHO,CHI,2015-02-25,12,1,2015,NaN,NaN,NaN,NaN,...,2.54,107.5,104.3,3.2,92.8,0.304,0.269,0.536,2015.0,CHI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87061,BOS,ATL,1979-10-27,5,1,1980,Boston Celtics,82.0,242.4,41.9,...,2.83,105.2,102.3,2.9,98.9,0.376,0.011,0.523,1980.0,ATL
87062,BOS,ATL,1980-02-26,11,1,1980,Boston Celtics,82.0,242.4,41.9,...,2.83,105.2,102.3,2.9,98.9,0.376,0.011,0.523,1980.0,ATL
87063,BOS,ATL,1980-03-14,-1,2,1980,Boston Celtics,82.0,242.4,41.9,...,2.83,105.2,102.3,2.9,98.9,0.376,0.011,0.523,1980.0,ATL
87064,PHI,ATL,1979-10-31,-5,2,1980,Philadelphia 76ers,82.0,242.1,42.0,...,2.83,105.2,102.3,2.9,98.9,0.376,0.011,0.523,1980.0,ATL


In [9]:
actual = actual.dropna()

In [10]:
cols = ['Season_FG_x',
        'Season_FGA_x', 'Season_FG%_x', 'Season_3P_x', 'Season_3PA_x',
        'Season_3P%_x', 'Season_2P_x', 'Season_2PA_x', 'Season_2P%_x',
        'Season_FT_x', 'Season_FTA_x', 'Season_FT%_x', 'Season_ORB_x',
        'Season_DRB_x', 'Season_TRB_x', 'Season_AST_x', 'Season_STL_x',
        'Season_BLK_x', 'Season_TOV_x', 'Season_PF_x', 'Season_PTS_x',
        'Season_Age_x', 'Season_MOV_x', 'Season_SOS_x',
        'Season_SRS_x', 'Season_ORtg_x', 'Season_DRtg_x', 'Season_NRtg_x',
        'Season_Pace_x', 'Season_FTr_x', 'Season_3PAr_x', 'Season_TS%_x',

        'Season_FG_y',
        'Season_FGA_y', 'Season_FG%_y', 'Season_3P_y', 'Season_3PA_y',
        'Season_3P%_y', 'Season_2P_y', 'Season_2PA_y', 'Season_2P%_y',
        'Season_FT_y', 'Season_FTA_y', 'Season_FT%_y', 'Season_ORB_y',
        'Season_DRB_y', 'Season_TRB_y', 'Season_AST_y', 'Season_STL_y',
        'Season_BLK_y', 'Season_TOV_y', 'Season_PF_y', 'Season_PTS_y',
        'Season_Age_y', 'Season_MOV_y', 'Season_SOS_y',
        'Season_SRS_y', 'Season_ORtg_y', 'Season_DRtg_y', 'Season_NRtg_y',
        'Season_Pace_y', 'Season_FTr_y', 'Season_3PAr_y', 'Season_TS%_y', ]

In [11]:
e = pd.DataFrame()

for i in ['Season_FG_x',
          'Season_FGA_x', 'Season_FG%_x', 'Season_3P_x', 'Season_3PA_x',
          'Season_3P%_x', 'Season_2P_x', 'Season_2PA_x', 'Season_2P%_x',
          'Season_FT_x', 'Season_FTA_x', 'Season_FT%_x', 'Season_ORB_x',
          'Season_DRB_x', 'Season_TRB_x', 'Season_AST_x', 'Season_STL_x',
          'Season_BLK_x', 'Season_TOV_x', 'Season_PF_x', 'Season_PTS_x',
          'Season_Age_x', 'Season_MOV_x', 'Season_SOS_x',
          'Season_SRS_x', 'Season_ORtg_x', 'Season_DRtg_x', 'Season_NRtg_x',
          'Season_Pace_x', 'Season_FTr_x', 'Season_3PAr_x', 'Season_TS%_x', ]:
    e[f'{i[:-2]}_Diff'] = actual[f'{i[:-2]}_x'] - actual[f'{i[:-2]}_y']
e

,Season_FG_Diff,Season_FGA_Diff,Season_FG%_Diff,Season_3P_Diff,Season_3PA_Diff,Season_3P%_Diff,Season_2P_Diff,Season_2PA_Diff,Season_2P%_Diff,Season_FT_Diff,...,Season_MOV_Diff,Season_SOS_Diff,Season_SRS_Diff,Season_ORtg_Diff,Season_DRtg_Diff,Season_NRtg_Diff,Season_Pace_Diff,Season_FTr_Diff,Season_3PAr_Diff,Season_TS%_Diff
0,-0.7,-6.3,0.025,1.9,2.7,0.045,-2.7,-9.0,0.025,3.1,...,-12.32,0.28,-12.04,-7.6,5.7,-13.3,-1.6,-0.070,-0.029,-0.028
1,-0.7,-6.3,0.025,1.9,2.7,0.045,-2.7,-9.0,0.025,3.1,...,-12.32,0.28,-12.04,-7.6,5.7,-13.3,-1.6,-0.070,-0.029,-0.028
2,-0.7,-6.3,0.025,1.9,2.7,0.045,-2.7,-9.0,0.025,3.1,...,-12.32,0.28,-12.04,-7.6,5.7,-13.3,-1.6,-0.070,-0.029,-0.028
7,0.3,-4.7,0.028,1.7,3.0,0.033,-1.6,-7.7,0.034,1.5,...,-8.72,0.31,-8.41,-5.9,3.4,-9.3,1.0,-0.074,-0.034,-0.012
8,0.3,-4.7,0.028,1.7,3.0,0.033,-1.6,-7.7,0.034,1.5,...,-8.72,0.31,-8.41,-5.9,3.4,-9.3,1.0,-0.074,-0.034,-0.012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87061,3.6,5.4,0.012,0.3,1.0,0.035,3.2,4.4,0.014,-3.5,...,4.88,-0.33,4.54,4.2,-0.4,4.6,3.7,-0.044,0.046,0.027
87062,3.6,5.4,0.012,0.3,1.0,0.035,3.2,4.4,0.014,-3.5,...,4.88,-0.33,4.54,4.2,-0.4,4.6,3.7,-0.044,0.046,0.027
87063,3.6,5.4,0.012,0.3,1.0,0.035,3.2,4.4,0.014,-3.5,...,4.88,-0.33,4.54,4.2,-0.4,4.6,3.7,-0.044,0.046,0.027
87064,3.7,8.4,-0.003,0.3,1.2,0.020,3.3,7.2,0.000,-4.4,...,1.31,-0.09,1.21,-0.2,-1.3,1.1,4.1,-0.036,0.006,0.021


In [12]:
X = e.copy()
y = actual['W/L']

In [13]:
# ...
# X = X.drop('Season_MOV_Diff', axis=1)
X.columns

Index(['Season_FG_Diff', 'Season_FGA_Diff', 'Season_FG%_Diff',
       'Season_3P_Diff', 'Season_3PA_Diff', 'Season_3P%_Diff',
       'Season_2P_Diff', 'Season_2PA_Diff', 'Season_2P%_Diff',
       'Season_FT_Diff', 'Season_FTA_Diff', 'Season_FT%_Diff',
       'Season_ORB_Diff', 'Season_DRB_Diff', 'Season_TRB_Diff',
       'Season_AST_Diff', 'Season_STL_Diff', 'Season_BLK_Diff',
       'Season_TOV_Diff', 'Season_PF_Diff', 'Season_PTS_Diff',
       'Season_Age_Diff', 'Season_MOV_Diff', 'Season_SOS_Diff',
       'Season_SRS_Diff', 'Season_ORtg_Diff', 'Season_DRtg_Diff',
       'Season_NRtg_Diff', 'Season_Pace_Diff', 'Season_FTr_Diff',
       'Season_3PAr_Diff', 'Season_TS%_Diff'],
      dtype='object')

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, )

In [ ]:

pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('feature_selection', SelectFromModel(estimator=RandomForestClassifier(),)),
    ('classifier', RandomForestClassifier())
])

param_grid = {
    'scaler': [
        StandardScaler(),
        MinMaxScaler(),
        MaxAbsScaler(),
        RobustScaler(),
        PowerTransformer(),
        QuantileTransformer(),
        Normalizer()
    ],
    'feature_selection__threshold': [
        'mean',
        'median',
        '1.25*mean'
    ],  # Threshold value or string
    'classifier': [
        RandomForestClassifier(n_estimators=500),
        # LogisticRegression(max_iter=500),
        # DecisionTreeClassifier()
    ]

}

# Randomized Search CV
# search = RandomizedSearchCV(pipeline, param_grid, cv=2, scoring='accuracy', verbose=5)
search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=5)
search.fit(X_train, y_train)



Fitting 5 folds for each of 21 candidates, totalling 105 fits
[CV 1/5] END classifier=RandomForestClassifier(n_estimators=500), feature_selection__threshold=mean, scaler=StandardScaler();, score=0.591 total time=  46.4s


In [ ]:
# Step 5: Get the best estimator and best parameters
best_estimator = search.best_estimator_
best_parameters = search.best_params_
best_score = search.best_score_

print("Best Estimator:", best_estimator, '\n')
print("Best Parameters:", best_parameters, '\n')
print("Best Score:", best_score, '\n')

In [ ]:
# feature_importances = best_estimator.named_steps['feature_selection'].estimator_.feature_importances_

# Create a DataFrame with feature names and importances
feature_importances_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': search.feature_importances_})

# Sort the DataFrame by importance in descending order
feature_importances_df = feature_importances_df.sort_values('Importance', ascending=False)

# Create a bar plot of feature importances using Seaborn
plt.figure(figsize=(14, 7))
sns.barplot(x='Importance', y='Feature', data=feature_importances_df.head(20), orient='h')
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Pipeline Feature Importance')
plt.show()

In [ ]:

# Step 4: Train the pipeline
pipeline.set_params(**best_parameters)
pipeline.fit(X_test, y_test)


In [ ]:

# Step 5: Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Step 6: Evaluate the performance of the pipeline
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
import joblib

# Save the pipeline
joblib.dump(pipeline, 'pipeline.pkl')